In [ ]:
import joblib
from io import BytesIO
from azure.storage.filedatalake import DataLakeServiceClient
import pandas as pd
import requests
import tweepy
import urllib.parse
from datetime import datetime, timedelta

In [14]:
# ─────────────────────────────────────────────────────────────────────────────
# 3) RECENT TWEETS VIA TWEEPY (last 30 days)
# ─────────────────────────────────────────────────────────────────────────────
encoded_token = "AAAAAAAAAAAAAAAAAAAAAAAs1AEAAAAAwSqw%2FQMhvxHeHOKIT2H5ro%2Ffcfw%3DdBWrxm1gxUhHRX8maJfxo1uBB4vAele6eqLKexEdz99ntbM8YQ"
TWITTER_BEARER_TOKEN = urllib.parse.unquote(encoded_token)
if not TWITTER_BEARER_TOKEN:
    raise ValueError("Set TWITTER_BEARER_TOKEN env var first")

client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN, wait_on_rate_limit=True)
query = "(bitcoin OR btc OR #bitcoin OR #btc) lang:en -is:retweet"

# Tweepy recent search only returns up to 7 days on standard API.
# If you have Academic access, use `search_all_tweets` instead.
start_time = thirty_ago.isoformat(timespec="seconds") + "Z"

paginator = tweepy.Paginator(
    client.search_recent_tweets,
    query,
    tweet_fields=["created_at","text"],
    start_time=start_time,
    max_results=100
)

tweets = []
for page in paginator:
    if page.data:
        tweets.extend(page.data)
    else:
        break

df_tweet = pd.DataFrame([
    {
      "date": t.created_at.date(),
      "text": t.text
    }
    for t in tweets
])
df_tweet["year"] = pd.to_datetime(df_tweet["date"]).dt.year

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 2) Aggregate tweet by day
# ─────────────────────────────────────────────────────────────────────────────
tweet_agg = (
    df_tweet
      .groupby("date")
      .agg(tweet_sentiment=("sentiment_score", "mean"),
           tweet_count    =("sentiment_score", "size"))
      .reset_index()
)